# Construção de uma Gramática Independente de Contexto

Trabalho desenvolvido por Rui Gonçalves (A101759) a 2024-03-15

## Descrição

Contrução de uma gramática para um prgrama na seguinte linguagem de programação:

```
?a
b = 2 * a - 3 / 6
!b
```

## Trabalho desenvolvido

### Gramática

G = (T, N, P, S)

- Terminais: $ T = \{ '?', '!', '=', '+', '-', '*', '/', '(', ')', id, number, '\$' \} $
- Não-Terminais: $ N = \{ Program, Statement, InputStatement, OutputStatement, AssignementStatement, Expression, PrimaryExpression, Factor, Factor', Term, Term', ParenthesizedExpression \} $
- Inicial: $ S = Program $
- Regras (P):

#### Program

- $ \text{Program} \rightarrow \text{Statement} \, \text{Program} $
- $ \text{Program} \rightarrow \text{'\$'} $

#### Statement

- $ \text{Statement} \rightarrow \text{InputStatement} $
- $ \text{Statement} \rightarrow \text{OutputStatement} $
- $ \text{Statement} \rightarrow \text{AssignementStatement} $

##### InputStatement

- $ \text{InputStatement} \rightarrow \text{'?'} \, \text{id} $

##### OutputStatement

- $ \text{OutputStatement} \rightarrow \text{'!'} \, \text{Expression} $

##### AssignmentStatement

- $ \text{OutputStatement} \rightarrow \text{id} \, \text{'='} \, \text{Expression} $

#### Expression

- $ \text{Expression} \rightarrow \text{Term} $

##### PrimaryExpression

- $ \text{PrimaryExpression} \rightarrow number $
- $ \text{PrimaryExpression} \rightarrow id $
- $ \text{PrimaryExpression} \rightarrow \text{ParenthesizedExpression} $

##### Factor

- $ \text{Factor} \rightarrow \text{PrimaryExpression} $
- $ \text{Factor} \rightarrow \text{PrimaryExpression} \, \text{'+'} \, \text{Term} $
- $ \text{Factor} \rightarrow \text{PrimaryExpression} \, \text{'-'} \, \text{Term} $

O que para ser LL(1), se tranforma em:

- $ \text{Factor} \rightarrow \text{PrimaryExpression} \, \text{Factor'} $

- $ \text{Factor'} \rightarrow \epsilon $
- $ \text{Factor'} \rightarrow \text{'+'} \, \text{Term} $
- $ \text{Factor'} \rightarrow \text{'-'} \, \text{Term} $

##### Term

- $ \text{Term} \rightarrow \text{Factor} $
- $ \text{Term} \rightarrow \text{Factor} \, \text{'*'} \, \text{Expression} $
- $ \text{Term} \rightarrow \text{Factor} \, \text{'/'} \, \text{Expression} $

O que para ser LL(1), se tranforma em:

- $ \text{Term} \rightarrow \text{Factor} \, \text{Term'} $

- $ \text{Term'} \rightarrow \epsilon $
- $ \text{Term'} \rightarrow \text{'*'} \, \text{Expression} $
- $ \text{Term'} \rightarrow \text{'/'} \, \text{Expression} $

##### ParenthesizedExpression

- $ \text{ParenthesizedExpression} \rightarrow \text{'('} \, \text{Expression} \, \text{')'} $

#### Lookahead

Apenas será necessário calcular o Lookahead das regras $ \text{Program} $, $ \text{Statement} $, $ \text{PrimaryExpression} $, $ \text{Factor'} $ e $ \text{Term'} $, pois estas têm multiplas definições, sendo este calculo necessário para provar que se trata de uma gramática LL(1).

##### PrimaryExpression

- $ L_{A}(PrimaryExpression_{1}) = \{ number \} $
- $ L_{A}(PrimaryExpression_{2}) = \{ id \} $
- $ L_{A}(PrimaryExpression_{3}) = L_{A}(ParenthesizedExpression) = \{ '(' \} $
- $ L_{A}(PrimaryExpression_{1}) \cap L_{A}(PrimaryExpression_{2}) \cap L_{A}(PrimaryExpression_{3}) = \{ number \} \cap \{ id \} \cap \{ '(' \} = \{\} $

##### Factor'

- $ L_{A}(Factor'_{1}) = Follow(Factor') = \{ id, '?', '!', '\$', '*', '/' \} $
- $ L_{A}(Factor'_{2}) = \{ '+' \} $
- $ L_{A}(Factor'_{3}) = \{ '-' \} $
- $ L_{A}(Factor'_{1}) \cap L_{A}(Factor'_{2}) \cap L_{A}(Factor'_{3}) = \{ id, '?', '!', '\$', '*', '/' \} \cap \{ '+' \} \cap \{ '-' \} = \{\} $

##### Term'

- $ L_{A}(Term'_{1}) = Follow(Term') = \{ id, '?', '!', '\$' \} $
- $ L_{A}(Term'_{2}) = \{ '*' \} $
- $ L_{A}(Term'_{3}) = \{ '/' \} $
- $ L_{A}(Term'_{1}) \cap L_{A}(Term'_{2}) \cap L_{A}(Term'_{3}) = \{ id, '?', '!', '\$' \} \cap \{ '*' \} \cap \{ '/' \} = \{\} $

##### Statement

- $ L_{A}(Statement_{1}) = L_{A}(InputStatement) = \{ '?' \} $
- $ L_{A}(Statement_{2}) = L_{A}(OutputStatement) = \{ '!' \} $
- $ L_{A}(Statement_{3}) = L_{A}(AssignementStatement) = \{ id \} $
- $ L_{A}(Statement_{1}) \cap L_{A}(Statement_{2}) \cap L_{A}(Statement_{3}) = \{ '?' \} \cap \{ '!' \} \cap \{ id \} = \{\} $

##### Program

- $ L_{A}(Program_{1}) = L_{A}(Statement) = \{ id, '?', '!' \} $
- $ L_{A}(Program_{2}) = \{ '\$' \} $
- $ L_{A}(Program_{1}) \cap L_{A}(Program_{2}) = \{ id, '?', '!' \} \cap \{ '\$' \} = \{\} $


### Processamento

Vamos então implementar um lexer para o problema.
Este foi implementado manualmente ao invés da utilização do Ply, por alguns problemas encontrados na utilização do mesmo no Jupyter.

In [1]:
import re

token_regex = [
    ('NUMBER', r'\d+(?:.\d+)?'),
    ('IDENTIFIER', r'[a-zA-Z_]\w*'),
    ('PLUS', r'\+'),
    ('MINUS', r'\-'),
    ('DIVIDE', r'/'),
    ('MULTIPLY', r'\*'),
    ('ASSIGN', r'='),
    ('QUESTION_MARK', r'\?'),
    ('EXCLAMATION_MARK', r'!'),
    ('SEMICOLON', r';'),
]

regex_combined = '|'.join('(?P<%s>%s)' % pair for pair in token_regex)


def num(s):
    try:
        return int(s)
    except ValueError:
        return float(s)


def lex_string(input_string):
    for mo in re.finditer(regex_combined, input_string):
        token_type = mo.lastgroup
        token_value = mo.group()

        if token_type == 'NUMBER':
            token_value = num(token_value)

        yield token_type, token_value

    yield '$', '$'

Definição da AST seguindo a gramática.

In [2]:
class Program:
    def __init__(self, statements):
        self.statements = statements

    def __repr__(self):
        return '\n'.join(map(str, self.statements))


class Statement:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return str(self.value)


class InputStatement:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return f'?{self.value}'


class OutputStatement:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return f'!{self.value}'


class AssignementStatement:
    def __init__(self, id, expression):
        self.id = id
        self.expression = expression

    def __repr__(self):
        return f'{self.id} = {self.expression}'


class Expression:
    def __init__(self, term):
        self.term = term

    def __repr__(self):
        return str(self.term)


class PrimaryExpression:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return str(self.value)


class Factor:
    def __init__(self, primary_expression, operator=None, factor=None):
        self.primary_expression = primary_expression
        self.operator = operator
        self.factor = factor


def __repr__(self):
    if self.factor:
        assert self.operator
        return f'{self.primary_expression} {self.operator} {self.factor}'
    return str(self.primary_expression)


class Term:
    def __init__(self, factor, operator=None, term=None):
        self.factor = factor
        self.operator = operator
        self.term = term

    def __repr__(self):
        if self.term:
            assert self.operator
            return f'{self.factor} {self.operator} {self.term}'
        return str(self.factor)


class ParenthesizedExpression:
    def __init__(self, expression):
        self.expression = expression

    def __repr__(self):
        return f'({self.expression})'

Vamos então implemenetar um parser, seguindo a gramática.

In [3]:
class Parser:
    def __init__(self, lexer):
        self.lexer = iter(lexer)
        self.lookahead = next(self.lexer)

    def parse(self):
        return self.program()

    def match(self, token_type):
        if self.lookahead[0] == token_type:
            self.lookahead = next(self.lexer)
        else:
            raise SyntaxError(f'Esperado {token_type}, encontrado {self.lookahead[0]}')

    def program(self):
        statements = []

        while self.lookahead[0] != '$':
            statements.append(self.statement())

        return Program(statements)

    def statement(self):
        statement = None
        
        if self.lookahead[0] == 'QUESTION_MARK':
            statement = self.input_statement()
        elif self.lookahead[0] == 'EXCLAMATION_MARK':
            statement = self.output_statement()
        elif self.lookahead[0] == 'IDENTIFIER':
            statement = self.assignment_statement()
            
        return statement

    def input_statement(self):
        self.match('QUESTION_MARK')
        value = self.lookahead[1]
        self.match('IDENTIFIER')
        return InputStatement(value)

    def output_statement(self):
        self.match('EXCLAMATION_MARK')
        value = self.expression()
        return OutputStatement(value)

    def assignment_statement(self):
        identifier = self.lookahead[1]
        self.match('IDENTIFIER')
        self.match('ASSIGN')
        expression = self.expression()
        return AssignementStatement(identifier, expression)

    def expression(self):
        return self.term()

    def primary_expression(self):
        primary = None
        
        if self.lookahead[0] == 'NUMBER':
            value = self.lookahead[1]
            self.match('NUMBER')
            primary = PrimaryExpression(value)
        elif self.lookahead[0] == 'IDENTIFIER':
            value = self.lookahead[1]
            self.match('IDENTIFIER')
            primary = PrimaryExpression(value)
        elif self.lookahead[0] == 'LPAREN':
            primary = self.parenthesized_expression()
            
        return primary

    def factor(self):
        primary_expression = self.primary_expression()
        factor_expression = None

        if self.lookahead[0] in ('PLUS', 'MINUS'):
            operator = self.lookahead[1]
            self.match(self.lookahead[0])
            factor = self.term()
            factor_expression = Factor(primary_expression, operator, factor)
        else:
            factor_expression = Factor(primary_expression)
            
        return factor_expression

    def term(self):
        factor = self.factor()
        term_expression = None

        if self.lookahead[0] in ('MULTIPLY', 'DIVIDE'):
            operator = self.lookahead[1]
            self.match(self.lookahead[0])
            term = self.expression()
            term_expression = Term(factor, operator, term)
        else:
            term_expression = Term(factor)
            
        return term_expression
    
    def parenthesized_expression(self):
        self.match('LPAREN')
        expression = self.expression()
        self.match('RPAREN')
        return ParenthesizedExpression(expression)

Vamos, por ultimo, então criar uma forma para fazermos um "*pretty-print*" de uma AST desta linguagem.

In [4]:
def pretty_print_ast(node, indent=0):
    if isinstance(node, list):
        for child in node:
            pretty_print_ast(child, indent)
    elif isinstance(node, str):
        print(' ' * indent + node)
    elif isinstance(node, int):
        print(' ' * indent + str(node))
    elif isinstance(node, float):
        print(' ' * indent + str(node))
    elif node is None:
        pass
    else:
        print(' ' * indent + node.__class__.__name__)
        
        for k, v in node.__dict__.items():
            pretty_print_ast(v, indent + 4)

### Exemplo

In [5]:
example = """
?a
b = 2 * a - 3 / 6
!b
"""

lexer = lex_string(example)
parser = Parser(lexer)
ast = parser.parse()

pretty_print_ast(ast)

Program
    InputStatement
        a
    AssignementStatement
        b
        Term
            Factor
                PrimaryExpression
                    2
            *
            Term
                Factor
                    PrimaryExpression
                        a
                    -
                    Term
                        Factor
                            PrimaryExpression
                                3
                        /
                        Term
                            Factor
                                PrimaryExpression
                                    6
    OutputStatement
        Term
            Factor
                PrimaryExpression
                    b


### Trabalho Futuro

Algumas sugestões de futuro trabalho:

- Lidar com erros no Lexer
- Colocação da informação de linhas/colunas no parser e em erros do parser
- Criação de um Visitor para a AST
- Compilação para uma linguagem intermédia (e.g. LLVM IR)
- Criação de um interpretador